In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import csv

In [4]:
data={'Price:':[],
      'Bedrooms:':[],
      'Full Baths:':[],
      'Half Baths:':[],
      'Three Quarter Baths:':[],
      'Square Footage:':[],
      'Acreage:':[],
      'Year Built:':[],
      'Listing ID #:':[],
      'Street Address:':[],
      'City:':[],
      'State:':[],
      'Postal Code:':[],
      'County:':[],
      'Community:':[],
      'Subdivision:':[],
      'Listing Status:':[],
      'Listing Type:':[],
     }

#### There are 168 pages.I use 20 pages only so that entire pages take a long time!

In [5]:
main_url='https://www.yourseattleareahomesearch.com'
all_house_data=[]
for i in range(1,20):
    url=f'https://www.yourseattleareahomesearch.com/search/quick?soldproperty=0&sortby=latest&type=quick&perpage=12&page={i}'
    html_data=requests.get(url).text
    soup=BeautifulSoup(html_data,'lxml')
    house_list=soup.find_all('div',class_='card--list js-searchresult')
    
    for house in house_list:
        url_house=f"{main_url}{house.a['href']}"
        html_house=requests.get(url_house).text
        soup_house=BeautifulSoup(html_house,'lxml')
        house_info=soup_house.find_all('ul',class_='flex pl-0')
        data_house={}
        for info in house_info:
        
            inf_type=info.find('li',class_='w-3 font-bold text-sm md:text-base text-bluegray-600 p-1 no-bullets snapshot-key').text.strip()        
            inf=info.find('li',class_='flex-1 w-9 text-sm md:text-base text-bluegray-600 p-1 no-bullets snapshot-value')
            data_house[inf_type]=inf.text      
        all_house_data.append(data_house) 

In [18]:
csv_file_path = 'output.csv'

with open(csv_file_path, 'w', newline='') as csvfile:
    fieldnames = all_house_data[3].keys()
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for item in all_house_data:
        writer.writerow(item)

In [19]:
df=pd.read_csv('output.csv')
df.columns=[col.replace(':','') for col in df.columns]

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228 entries, 0 to 227
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Price                228 non-null    object 
 1   Bedrooms             214 non-null    float64
 2   Full Baths           206 non-null    float64
 3   Half Baths           114 non-null    float64
 4   Three Quarter Baths  124 non-null    float64
 5   Square Footage       218 non-null    object 
 6   Acreage              212 non-null    float64
 7   Year Built           220 non-null    float64
 8   Listing ID #         228 non-null    object 
 9   Street Address       228 non-null    object 
 10  City                 228 non-null    object 
 11  State                228 non-null    object 
 12  Postal Code          228 non-null    int64  
 13  County               228 non-null    object 
 14  Community            228 non-null    object 
 15  Subdivision          228 non-null    obj

In [24]:
df.head()

,Price,Bedrooms,Full Baths,Half Baths,Three Quarter Baths,Square Footage,Acreage,Year Built,Listing ID #,Street Address,City,State,Postal Code,County,Community,Subdivision,Listing Status,Listing Type
0,"$1,100,000",4.0,2.0,NaN,NaN,"2,040",0.09,1927.0,NWM2158715,7534 18th Ave NE,Seattle,WA,98115,King,710 - North Seattle,Ravenna,Active,Residential
1,"$520,000",3.0,1.0,NaN,1.0,"1,364",0.25,1970.0,NWM2159727,5316 Bellaire Way,Bellingham,WA,98226,Whatcom,865 - Meridian,Bellingham,Active,Residential
2,"$599,900",4.0,2.0,1.0,NaN,"2,028",0.29,2004.0,NWM2159457,818 W Sage Bay Dr,Moses Lake,WA,98837,Grant,297 - Central Moses Lake,Sage Bay,Active,Residential
3,"$1,350,000",3.0,1.0,1.0,1.0,"2,360",0.17,1960.0,NWM2158982,9244 20th Ave NW,Seattle,WA,98117,King,705 - Ballard/Greenlake,Olympic Manor,Active,Residential
4,"$710,000",1.0,1.0,NaN,1.0,"1,212",3.00,2013.0,NWM2158653,200 Quad Dr,Cle Elum,WA,98922,Kittitas,948 - Upper Kittitas County,Granite Creek,Active,Residential
